# Revision notes for incorrectly answered questions
The following are notes from questions which I got wrong in PCAP prep

## Python packages
The following are true with regard to Python packages:
- The shebang statement (`#!`) which appears at the top of files is a Unix command for how the file should be handled and has no effect on MS Windows.
- If you want to use a packages in a non-standard directory you should append the directory's path to the path variable using the `sys` module.
- Packages *should* have an `__init__.py`, however, they __will__ work without one.
- During the first import of a module Python translates its source code into a *__semi__-compiled* format stored in the `.pyc` files and deploys these files to the `__pycache__` directory located in the module's home directory.

## Exception handling
Look at the following code:

In [18]:
class Err(Exception):
    def __init__(self, msg):
        self.message = msg
        
    def __str__(self):
        return "From Err block"
    
try:
    print("Start")
    raise Exception ("Error raised")
except Err as e:
    print(e)
else:
    print("From else block")

Start


Exception: Error raised

It will print `Start` and then raise an *unhandled* exception because:
- the `except` branch is only handling `Err` class exceptions whereas the exception we raised is `Exception` class.

If we revised line 10 to `raise Err("Error raised")` we would now handle the exception, however, the message printed would be `From Err block` as that is explicitly defined as the `__str__` method of the `Err` class. If we deleted the `__str__` method from the `Err` class, *then* we would get `Error raised` as the output.

In [20]:
class Err(Exception):
    def __init__(self, msg):
        self.message = msg
        
#     def __str__(self):
#         return "From Err block"
    
try:
    print("Start")
    raise Err ("Error raised")
except Err as e:
    print(e)
else:
    print("From else block")

Start
Error raised


## `assert "zero"`
Look at the following code:

In [22]:
try:
    assert "zero"
except:
    print("one", end=" ")
else:
    print("two", end=" ")
finally:
    print("three")

two three


It will print:
<br>`two three`

Because:
- `assert` only raises an error if the value being checked is `False` (i.e. empty strings, empty lists, False, 0 etc).
- a non-empty string evaluates as `True`, as such no assertion error is raised and the executing is `try`>`else`>`finally`.

## `int` divided by `int` equals `float`
Look at the following code:

In [24]:
string_convert = str(16**1/2)

new_string = ""

for ch in string_convert:
    new_string += ch
    
print(new_string[-1])

0


It will ouput `0` because:
- Brackets have priority so the maths calculation is done first, then __PEMDAS__ has exponentiation executed first so:
    - 16**1 = 16
    - 16 / 2 = 8<b>.0</b> (remember integer divided by integer is always a float)
- The `for` loop adds `8.0` to the `new_string` string
- Index `-1` prints the final character of the string which is `0`

## An object instance changing a class variable
Look at the following code:

In [25]:
class MyClass:
    c_var = 0
    
    def __init__(self):
        self.value = 0
        
obj_a = MyClass()
obj_a.c_var = 5
obj_b = MyClass()
print(obj_a.c_var, obj_b.c_var)

5 0


It will output `5 0` because:
- `c_var` is a class variable (aka a static variable) which can be accessed using `MyClass.c_var`. 
- `obj_a` and `obj_b` are both instances of `MyClass` and thus can __read__ the `c_var` variable
- Python will first look within the object(s) for a particular var and if it's not present will then look in class from which the objects are derived. However, the class variable can only be __read__ via the object instances not changed.
- Since the object instances cannot write to the class variable Python will instead create a new local variable within the object. So `obj_a.c_var` no longer points to the class variable `c_var` but a new *local* var of the same name within the `obj_a` instance. Thus printing `obj_a.c_var` will print the local var value of `5`
- In the meantime, `obj_b` hasn't tried to write to `c_var` so it will still point to the original class var which is still `0`

## The `__name__` class property
Look at the following code:

In [27]:
class NewClass:
    pass

obj = NewClass()

print(NewClass.__name__)
print(type(obj).__name__)

NewClass
NewClass


Both `print`s will output the class name `NewClass`.
- The `__name__` property is a built in __class__ property
- In order to print the class name using an object instance we need to use the `type()` function which will return the class name and thus the second print is essentially access the same `NewClass.__name__` property.
- If we were to try printing `obj.__name__` directly, Python would raise an `AttributeError` as objects do not have a `__name__` property.

## The `__module__` property
Look at the following code:

In [30]:
# inside my_module.py

class AnotherClass:
    pass

print(AnotherClass.__module__)

__main__


It will output `__main__` because:
- We can see that no `import` statements are present which means the `AnotherClass` class was defined in the currently running script (`my_module.py`).
- Since we're currently running `my_module.py`, the `__module__` property will return `__main__` (this is the name Python gives to the currently running document.
- If we were running a different file and had imported `AnotherClass` from `my_module.py` *then* the output would be `my_module.py`

## Class variable inheritance
Look at the following code:

In [35]:
class One:
    pass

class Two(One):
    var = 1
    
class Three(One):
    var = 2
    
class Four(Two, Three):
    pass

four_obj = Four()

print(four_obj.var)

1


The code is __valid__ and will output `1`.
- This is a demonstration of the *diamond problem* of multiple inheritance. 
- Since classes `Two` and `Three` are both subclasses of `One` there is no __method resolution order__ (MRO) conflict and `Four` will inherit first from the left-most parent which is `Two`
- If `Three` had been a subclass of `Two` instead *then* we would have triggered a `TypeError` due to an MRO conflict.

## File handling
- The `open()` method returns an iterable object and this can be used to iterate through the data in the file
- The `readlines()` method returns an *empty* __list__ if the stream opened is an empty file
- `rb` is the `read, binary` mode
- `readinto()` is the method used to read from a binary file. It returns the number of successfully read bytes. The method will try to fill all available space in its argument. If there are more data than available space the operation will stop before the the end of the file.